Flood Analysis in Opole (2024) using Sentinel-1 SAR Data

In this exercise, you will compare radar backscatter from Sentinel-1 before and after the 2024 flood in Opole, Poland.

Objective:
- Load Sentinel-1 VV data before and after the flood
- Convert radar intensity to decibels (dB)
- Calculate the difference between the two dates
- Identify areas with a significant decrease in VV (possible flood zones)


In [9]:
import ee
import geemap.foliumap as geemap
import webbrowser
import ssl

ssl._create_default_https_context = ssl._create_stdlib_context
ee.Authenticate()
ee.Initialize(project='ee-juliaszymanska142')
Map = geemap.Map(center=[50.675, 17.931], zoom=12)
region = ee.Geometry.Point([17.931, 50.675])


before_flood = ee.ImageCollection('COPERNICUS/S1_GRD') \
    .filterBounds(region) \
    .filterDate('2024-09-01', '2024-09-14') \
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
    .select('VV') \
    .mean()

after_flood = ee.ImageCollection('COPERNICUS/S1_GRD') \
    .filterBounds(region) \
    .filterDate('2024-09-19', '2024-09-30') \
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
    .select('VV') \
    .mean()



In [10]:
before_dB = before_flood.log10().multiply(10)
after_dB = after_flood.log10().multiply(10)

In [11]:
diff_dB = before_dB.subtract(after_dB)
flood_mask = diff_dB.gt(3)

In [12]:
vis_params = {'min': -25, 'max': 0}
Map.addLayer(before_dB, vis_params, 'Before Flood (dB)')
Map.addLayer(after_dB, vis_params, 'After Flood (dB)')
Map.addLayer(diff_dB, {'min': -5, 'max': 5, 'palette': ['blue', 'white', 'red']}, 'Difference (dB)')
Map.addLayer(flood_mask.updateMask(flood_mask), {'palette': ['cyan']}, 'Potential Flooded Areas')

Map
